In [14]:
try:
    import cv2
    import numpy as np
    import mediapipe
    import matplotlib.pyplot as plt
    import pyautogui
    import mouse
    import traceback
    import math
    capture = cv2.VideoCapture(0)
    fw = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    fh = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)

    def show(i):
        if i is not None:
            coor = mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(i.x,i.y,fw,fh)
            if coor is not None:
                cv2.putText(frame,str(coor),(coor[0]+20,coor[1]+20),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0, 0), 2)

    with mediapipe.solutions.hands.Hands(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=1) as hands:
        while capture.isOpened():
            ret, frame = capture.read()
            if ret == False:
                continue
            frame = cv2.flip(frame, 1)
            results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if results.multi_hand_landmarks != None:
                thumb=results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.THUMB_TIP]
                index = results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
                middle = results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring= results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.RING_FINGER_TIP]
                little=results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.PINKY_TIP]
                a=[thumb,index,middle,ring,little]
                x0=0
                y0=0
                for i in a:
                    x0+=i.x
                    y0+=i.y
                c=(x0/5,y0/5)
                centroid = mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(c[0],c[1],fw,fh)
                frame=cv2.circle(frame, centroid, 10, (0, 0, 255), -1)
                dis=[math.dist(mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(i.x,i.y,fw,fh),centroid) for i in a]
                print(dis)
                show(thumb)
                show(index)
                show(middle)
                show(ring)
                show(little)
            cv2.imshow('Mouse Controller', frame)
            if cv2.waitKey(1) == 27: #escape key
                break
    cv2.destroyAllWindows()
    capture.release()

except:
    cv2.destroyAllWindows()
    capture.release()
    traceback.print_exc()

[84.50443775329198, 42.19004621945797, 27.018512172212592, 34.655446902326915, 71.70076708097342]
[83.24061508662703, 46.32493928760188, 29.017236257093817, 31.240998703626623, 71.1125867902441]
[84.20213774008353, 48.05205510693585, 30.06659275674582, 33.941125496954285, 73.17103251970687]
[90.60353193998564, 49.36598018878994, 35.014282800023196, 38.2099463490856, 77.02596964660685]
[88.23831367382311, 48.166378315169176, 36.013886210738214, 40.31128874149275, 77.02596964660685]
[88.86506625215559, 48.25971404805461, 36.0, 40.31128874149275, 77.02596964660685]
[89.64373932405988, 49.254441424099, 36.0, 39.59797974644666, 79.02531240052139]
[89.02246907382427, 50.24937810560445, 36.0, 39.59797974644666, 77.02596964660685]
[89.02246907382427, 50.24937810560445, 36.013886210738214, 39.59797974644666, 78.02563681252464]
[91.67878707749138, 51.62363799656123, 36.124783736376884, 39.698866482558415, 78.00640999302557]
[90.78546139112804, 51.478150704935004, 37.0, 38.94868418830089, 78.0256